In [3]:
# 新建ismn站点文件夹，文件夹下有多个子文件夹，名为站名
# 站名对应的文件夹下有
# 1、intro.txt文件，里面csv格式存储的有站点、经度、纬度
# 2、ismn.csv文件，里面存储着该站点的ismn数据
# 3、era5.nc文件，里面存储着该站点的era5数据
# 4、smap.h5文件，里面存储着该站点的smap数据
# 5、esacci.nc文件，里面存储着该站点的esacci数据

import os
import zipfile
import xarray as xr
import shutil
from tqdm import tqdm

# 定义输入文件夹和输出文件名
input_folder = r"D:\dataset\ERA5_yellow"
output_file = r"D:\dataset\ERA5_yellow\era5_ssm.nc"

# 获取所有.zip文件的列表
zip_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.zip')]

# 创建一个空的xarray数据集，用于存储合并后的数据
merged_data = None

# 遍历每个.zip文件并解压缩并合并其中的.nc文件
for zip_file in tqdm(zip_files, desc="Processing files"):
    # 删除现有的'tmp_unzip'文件夹（如果存在）
    if os.path.exists('tmp_unzip'):
        shutil.rmtree('tmp_unzip')
    
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # 解压缩.zip文件到临时文件夹
        zip_ref.extractall('tmp_unzip')
        
        # 获取解压后的.nc文件的路径
        nc_file_path = os.path.join('tmp_unzip', 'data.nc')
        
        # 打开.nc文件并将其合并到数据集中
        ds = xr.open_dataset(nc_file_path)
        if merged_data is None:
            merged_data = ds
        else:
            merged_data = xr.concat([merged_data, ds], dim='time')
        
        # 关闭.nc文件
        ds.close()
        
# 删除临时文件夹
shutil.rmtree('tmp_unzip')

# 将合并后的数据保存为新的.nc文件
merged_data.to_netcdf(output_file)

print(f'Merged data saved to {output_file}')


Processing files: 100%|██████████| 72/72 [01:54<00:00,  1.60s/it]


Merged data saved to D:\dataset\ERA5_yellow\era5_ssm.nc


In [5]:
nc = xr.open_dataset(r"D:\dataset\ERA5_yellow\era5_ssm.nc")
#取出各variable的数据看看,数据格式为numpy数组
for var in nc.variables.keys():
    data=nc.variables[var][:].data
    print(var,data.shape)
#     np.save(var+'.npy',data)

longitude (251,)
latitude (101,)
time (26298,)
skt (26298, 101, 251)
tp (26298, 101, 251)
swvl1 (26298, 101, 251)


In [ ]:
import numpy as np

In [ ]:
def get_gridCenter(degree,n):
    n1 = n/2
    degree = degree-np.sign(degree)*n1

    minm = np.sign(degree//n)*(abs(degree//n) * n + n1) 
    # print(minm)
    center = minm+n1

    return center

In [ ]:
# datadir = "H:\project\ERA5\\"
datadir = "F:\project\ERA5\\"
nan_points = []

# variables = ["skt","tp","swvl1"]
variables = {'skin_temperature':'skt','total_precipitation':'tp','volumetric_soil_water_layer_1':'swvl1'}
# variables = ["swvl1"]

# print(datadir+"area"+area)
for area,points_dir in area_dic.items():
    n = 0
    for j in glob.glob(datadir+"area"+str(area)+"\\*"):
        print(j)
        for var_key in variables.keys():
            if var_key in j:
                var_name = variables[var_key]
                break
        nc = xr.open_dataset(j)

        # print(nc)
        v = nc[var_name]  # prec为变量内容（降雨）
        for point_dir in points_dir:
            df = pd.read_csv(point_dir+"\\intro.txt")
            lat = float(df["lat"])
            lon = float(df["lon"])
            print(lat,lon)
            centerlat = get_gridCenter(lat,0.1)
            centerlon = get_gridCenter(lon,0.1)
            
            
            olat,olon = v.latitude.data,v.longitude.data
            centerlon_index,centerlat_index = np.argmin(abs(olon-centerlon)),np.argmin(abs(olat-centerlat))

            lon_range = olon[centerlon_index-3:centerlon_index+4]
            lat_range = olat[centerlat_index-3:centerlat_index+4]
            


            nc_30 = v.loc['2001-1-1':'2020-1-1']  # 截取时间
            # print(lon_range,lat_range)
            # print(centerlon,centerlat)

            # t1 = nc_30.interp(latitude = lat_range,longitude=lon_range)
            nc_coordinate = nc_30.sel(longitude=lon_range, latitude=lat_range)
            # print(len(nc_coordinate.longitude))

            new_lon = np.arange(nc_coordinate.longitude[0]-0.05,nc_coordinate.longitude[-1]+0.06,0.05)
            new_lat = np.arange(nc_coordinate.latitude[0]+0.05,nc_coordinate.latitude[-1]-0.06,-0.05)
            # new_lon = np.arange(lon_range[0],lon_range[-1]+0.05,0.05)
            # new_lat = np.arange(lat_range[0],lat_range[-1]-0.05,-0.05)


            nc1 = nc_coordinate.interp(longitude=new_lon,latitude=new_lat,method='nearest')
            if nc1.shape[1]<=9 or nc1.shape[2]<=9:
                print(nc1.shape,centerlon_index,len(olon),centerlat_index,len(olat))
                # n+=1
                break
            # grid_size = new_lon[1]-new_lon[0]

            # centerlat1,centerlon1 = get_gridCenter(lat,grid_size),get_gridCenter(lon,grid_size)
            # print(centerlat1,centerlon1)
            # print(new_lat,new_lon)
            # centerlat1,centerlon1 = new_lat[abs(new_lat-centerlat1).argmin()], new_lon[abs(new_lon-centerlon1).argmin()] 
            
            centerlat1,centerlon1 = abs(new_lat-lat).argmin(), abs(new_lon-lon).argmin()

            lon_range1 = new_lon[centerlon1-3:centerlon1+4]
            lat_range1 = new_lat[centerlat1-3:centerlat1+4]


            nc2 = nc1.sel(longitude=lon_range1,latitude=lat_range1)
            print(nc2.shape)
            if nc2.shape[1]!=7 or nc2.shape[2]!=7:
                print(nc2.shape,centerlon_index,len(olon),centerlat_index,len(olat))
                # n+=1
                continue
            elif nc2.isnull().sum().sum()>0:
                print(nc2.isnull().sum().sum())
                print(point_dir)
                nan_points.append(point_dir)
                # n+=1
                continue
            # n+=1
            # if nc2.loc['2012-01-01'][0].shape != (7,7):
            #     print(point_dir,'no')
            #     print(len(lon_range1),len(lat_range1))
            #     print(nc2.loc['2012-01-01'][0].shape)
                # break

            ds1 = xr.Dataset({var_name: nc2})
            # print(ds1.dims)
            last_dir = point_dir+"\\"+var_name+".nc"

            ds1.to_netcdf(last_dir)
            # print('save done')
        n+=1